# **Financial Applicactions with ML & AI**

<img style="float: right;" src="https://github.com/torreblanca99/course_financial_applications/blob/develop/docs/img/logo_bourbaki.png?raw=1" width="100"/>

## **Module II:** Value Risk
#### Topic: Delta Hedging

##### Name: Julio César Avila Torreblanca

- **Objective**: apply Delta Hedging
- **Contents**:
    - Notes:
        - Delta Hedging
    - Code:
        1. Libraries and parameters
        2. Examples with synthetic data
        3. Exercises with a real asset
----

# Delta Hedging

## Introduction

Delta hedging is a risk management strategy used in options trading to reduce or eliminate the directional risk associated with price movements of the underlying asset. Delta $(\Delta$) represents the rate of change in the option's price for a one-unit change in the price of the underlying asset.

## Purpose

The main goal of delta hedging is to create a delta-neutral position, where the overall delta of the portfolio equals zero. This means the portfolio is theoretically immune to small price movements in the underlying asset.

## Mathematical Representation

For an option on an underlying asset:
- **Delta ($\Delta$)** is expressed mathematically as:
  $$
  \Delta = \frac{\partial V}{\partial S}
  $$
  where $V$ is the value of the option and $S$ is the underlying asset price.

- **Gamma ($\Gamma$)** measures the rate of change of Delta with respect to changes in the underlying asset price and is given by:
  $$
  \Gamma = \frac{\partial^2 V}{\partial S^2}
  $$
  This represents how sensitive the delta is to movements in the underlying price.

## Delta Hedging Strategy

1. **Calculate Delta:**
   Determine the delta of the option position.

2. **Establish a Hedge:**
   Adjust the position by buying or selling the appropriate amount of the underlying asset to offset the option's delta.

3. **Rebalance Regularly:**
   As the price of the underlying asset changes and time to expiration decreases, the delta changes. Adjustments are required to maintain delta neutrality.

## Practical Example

Let's assume you own a call option on Alphabet Inc. (GOOGL):

- Current price of GOOGL: $2,700 per share.
- Delta of the call option: 0.6
- Number of call options: 10 (each contract typically represents 100 shares)

### Step-by-Step

1. **Calculate the Total Delta of the Options Position:**
   $$
   \text{Total Delta} = 0.6 \times 10 \times 100 = 600
   $$

2. **Create a Delta-Neutral Portfolio:**
   To neutralize this delta, you need to short 600 shares of GOOGL.

3. **Monitor and Rebalance:**
   As GOOGL's price or the option's delta changes, you must adjust the hedge by buying or selling shares to maintain a delta-neutral position.

## Advantages

- **Risk Reduction:** Mitigates potential losses from small movements in the underlying asset's price.

## Disadvantages

- **Frequent Adjustments:** Requires continuous monitoring and trading, which incurs transaction costs.

- **Complexity and Imperfection:** While delta hedging addresses small price changes, it may not fully protect against larger movements or changes in implied volatility.

Delta hedging is fundamental for risk management in options trading, allowing traders to maintain portfolio value stability despite market volatility.

---

# 1. Libreries and parameters

In [1]:
import math
from scipy.stats import norm
import numpy as np

## 1.1 Functions

Next functions creates a portfolio in which the weights assigned to each asset will be taken as the inverse-variances (more secure inversions).

In [2]:
# Define una opción de tipo Call Europeo
class EuropeanCall:
    # Call delta
    def call_delta(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        return z1

    # Call gamma
    def call_gamma(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z2 = z1 / (asset_price * asset_volatility * math.sqrt(time_to_expiration))
        return z2

    def call_price(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z1 = z1 * asset_price
        x2 = math.log(asset_price / (b * strike_price)) - .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x2 = x2 / (asset_volatility * (time_to_expiration ** .5))
        z2 = norm.cdf(x2)
        z2 = b * strike_price * z2
        return z1 - z2

    def __init__(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.call_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.call_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.call_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

In [3]:
# Define una opción de tipo Put Europeo
class EuropeanPut:
    # Put delta
    def put_delta(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        return z1 - 1

    # Put gamma
    def put_gamma(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log(asset_price / (b * strike_price)) + .5 * (
                asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z2 = z1 / (asset_price * asset_volatility * math.sqrt(time_to_expiration))
        return z2

    def put_price(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = math.log((b * strike_price) / asset_price) + .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x1 = x1 / (asset_volatility * (time_to_expiration ** .5))
        z1 = norm.cdf(x1)
        z1 = b * strike_price * z1
        x2 = math.log((b * strike_price) / asset_price) - .5 * (
                    asset_volatility * asset_volatility) * time_to_expiration
        x2 = x2 / (asset_volatility * (time_to_expiration ** .5))
        z2 = norm.cdf(x2)
        z2 = asset_price * z2
        return z1 - z2

    def __init__(
            self, asset_price, asset_volatility, strike_price,
            time_to_expiration, risk_free_rate
    ):
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.put_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.call_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.put_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

# 2. Exmaple 1 with synthetic data

In [4]:
# asset_price : underlying asset price
# sigma       : implied volatility for the Black-Scholes model input
# dt          : time to expiration
# rf          : risk-free rate (e.g., treasury bonds or government bills, depending on the country)
# nContract1  : number of contracts
# K1          : strike price of option 1
# K2          : strike price of option 2
# K3          : strike price of option 3

# Underlying asset price
asset_price = 543

# Black-Scholes input
sigma = 0.53
dt = 30/365
rf = 0.015

# Option 1
nContract1 = -1000
K1 = 545

# Option 2
K2 = 540

# Option 3
K3 = 555

## 2.1 Compute call and greeks for the the fisrt option

In [5]:
# Position with a single option
option1 = EuropeanCall(
    asset_price=asset_price,
    asset_volatility=sigma,
    strike_price=K1,
    time_to_expiration=dt,
    risk_free_rate=rf
)

# Theoretical initial portfolio value:
print(f'Theoretical Initial Portfolio value: ${option1.price * abs(nContract1):,.1f}')

# Calculate and print portfolio Greeks (adjusted for the number of contracts)
print('Initial Portfolio Greeks:\n'
      'Delta: {}\n'
      'Gamma: {}'.format(option1.delta * nContract1, option1.gamma * nContract1))

Theoretical Initial Portfolio value: $32,264.1
Initial Portfolio Greeks:
Delta: -523.8788365375873
Gamma: -6.3495209433350475


- ***Delta*** represents the sensitivity of the option's price to changes in the price of the underlying asset. In other words, it tells you how much the option's price is expected to change for a one-unit change in the underlying asset's price. For example, if the delta is 0.6, a $1 increase in the stock price would theoretically result in a $0.60 increase in the option's price.

- ***Gamma*** quantifies the rate of change of delta with respect to changes in the underlying asset's price. It measures the curvature or acceleration in the option's price movement as the underlying price changes. A high gamma indicates that delta can change dramatically, meaning the option's price sensitivity is rapidly evolving with movements in the underlying asset. This is especially important for managing risk in dynamic hedging strategies.

## 2.2 Compute call and greeks for the other two options

In [6]:
# Price and Greeks for option2 and option3

option2 = EuropeanCall(
    asset_price=asset_price, 
    asset_volatility=sigma, 
    strike_price=K2, 
    time_to_expiration=dt, 
    risk_free_rate=rf
)

option3 = EuropeanCall(
    asset_price=asset_price, 
    asset_volatility=sigma, 
    strike_price=K3, 
    time_to_expiration=dt, 
    risk_free_rate=rf
)

# Print theoretical prices and Greeks for option2
print(f"Option 2 - Price: ${option2.price* abs(nContract1):,.1f} \n Delta: {option2.delta:.4f}\n Gamma: {option2.gamma:.4f}")

# Print theoretical prices and Greeks for option3
print(f"Option 3 - Price: ${option3.price* abs(nContract1):,.1f} \n Delta: {option3.delta:.4f}\n Gamma: {option3.gamma:.4f}")

Option 2 - Price: $34,638.0 
 Delta: 0.5480
 Gamma: 0.0066
Option 3 - Price: $27,873.4 
 Delta: 0.4762
 Gamma: 0.0058


## 2.3 Neutralize both Greeks

In [7]:
# OPTION 1
portfolio_greeks = [
    [option1.gamma * abs(nContract1)], 
    [option1.delta * abs(nContract1)]
]

portfolio_greeks

[[np.float64(6.3495209433350475)], [np.float64(523.8788365375873)]]

***Greek values for Option 1 (i.e., what is currently in our portfolio)***

$$
\begin{bmatrix} \Gamma \\ \Delta \end{bmatrix} = \begin{bmatrix}6.3495... \\  523.8788 \end{bmatrix}
$$

In [8]:
# OPTION 2 & 3 Greeks
greeks = np.array([
    [option2.gamma, option3.gamma], 
    [option2.delta, option3.delta]
    ])
greeks

array([[0.00664158, 0.00577126],
       [0.54797566, 0.47616875]])

In [9]:
print(f'Gamma option 2: {np.round(option2.gamma,4)}\n'
      f'Delta option 2: {np.round(option2.delta,4)}\n'
      f'Gamma option 3: {np.round(option3.gamma,4)}\n'
      f'Delta option 3: {np.round(option3.delta,4)}')
     

Gamma option 2: 0.0066
Delta option 2: 0.548
Gamma option 3: 0.0058
Delta option 3: 0.4762


We want to find the position we would have with respect to options 2 and 3, given their Greeks.

$$ \begin{bmatrix} 
    0.0.0066 & 0.0.0057\\ 
    0.0.548 & 0.4762 
    \end{bmatrix}
    \begin{bmatrix} 
    w_2 \\ 
    w_3 
    \end{bmatrix}
    =
    \begin{bmatrix} 
    6.3495 \\  
    523.8788
    \end{bmatrix} 
$$

$$ w = A^{-1} \cdot A \cdot w = A^{-1} \cdot P $$


How can we neutralize this?

In [10]:
np.round(portfolio_greeks, 5)

array([[  6.34952],
       [523.87884]])

In [11]:
# Neutralization of Greeks -- delta and gamma
inv = np.linalg.inv(np.round(greeks, 5))  # It is recommended to round to avoid issues with matrix inversion

# Positions in options 2 and 3 to neutralize delta and gamma
w = np.dot(inv, portfolio_greeks)

In [12]:
w

array([[-8845.65654317],
       [11279.84062217]])

In [13]:
greeks @ w # validate, this is P

array([[  6.34981499],
       [523.90309769]])

In [15]:
print(f'Final Positions: \n'
      f'Option 1: {nContract1} \n'
      f'Option 2: {w[0][0]} \n'
      f'Option 3: {w[1][0]}')

Final Positions: 
Option 1: -1000 
Option 2: -8845.656543165445 
Option 3: 11279.84062217176


To do: (Reduce loss)
- Option 1: sell 1000 shares
- Option 2: sell 8845 shares
- Option 3: buy 11279 shares

# 3. Exercise 

In [41]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import datetime as dt

In [47]:
class EuropeanCall:
    """
    Represents a European Call Option and provides methods to calculate its price, Delta, and Gamma.

    Attributes:
        asset_price (float): Current price of the underlying asset.
        asset_volatility (float): Volatility of the underlying asset.
        strike_price (float): Strike price of the option.
        time_to_expiration (float): Time to expiration in years.
        risk_free_rate (float): Risk-free interest rate.
        price (float): Price of the call option.
        delta (float): Delta of the call option.
        gamma (float): Gamma of the call option.
    """

    def __init__(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Initializes the European Call Option with its parameters and calculates its price, Delta, and Gamma.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.
        """
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.call_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.call_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.call_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

    def call_delta(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the Delta of the European Call Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Delta of the call option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        return norm.cdf(x1)

    def call_gamma(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the Gamma of the European Call Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Gamma of the call option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        return norm.pdf(x1) / (asset_price * asset_volatility * math.sqrt(time_to_expiration))

    def call_price(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the price of the European Call Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Price of the call option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        x2 = x1 - asset_volatility * math.sqrt(time_to_expiration)
        return (norm.cdf(x1) * asset_price) - (norm.cdf(x2) * b * strike_price)

    

class EuropeanPut:
    """
    Represents a European Put Option and provides methods to calculate its price, Delta, and Gamma.

    Attributes:
        asset_price (float): Current price of the underlying asset.
        asset_volatility (float): Volatility of the underlying asset.
        strike_price (float): Strike price of the option.
        time_to_expiration (float): Time to expiration in years.
        risk_free_rate (float): Risk-free interest rate.
        price (float): Price of the put option.
        delta (float): Delta of the put option.
        gamma (float): Gamma of the put option.
    """
    def __init__(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Initializes the European Put Option with its parameters and calculates its price, Delta, and Gamma.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.
        """
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.put_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.put_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.gamma = self.put_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

    def put_delta(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the Delta of the European Put Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Delta of the put option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        return norm.cdf(x1) - 1

    def put_gamma(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the Gamma of the European Put Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Gamma of the put option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        return norm.pdf(x1) / (asset_price * asset_volatility * math.sqrt(time_to_expiration))

    def put_price(self, asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate):
        """
        Calculates the price of the European Put Option.

        Args:
            asset_price (float): Current price of the underlying asset.
            asset_volatility (float): Volatility of the underlying asset.
            strike_price (float): Strike price of the option.
            time_to_expiration (float): Time to expiration in years.
            risk_free_rate (float): Risk-free interest rate.

        Returns:
            float: Price of the put option.
        """
        b = math.exp(-risk_free_rate * time_to_expiration)
        x1 = (math.log(asset_price / (b * strike_price)) +
              0.5 * (asset_volatility ** 2) * time_to_expiration) / (asset_volatility * math.sqrt(time_to_expiration))
        x2 = x1 - asset_volatility * math.sqrt(time_to_expiration)
        return (norm.cdf(-x2) * b * strike_price) - (norm.cdf(-x1) * asset_price)

    

In [61]:
# Select a stock ticker (e.g., GOOGL)
ticker = "GOOGL"

# Download historical data for the stock
data = yf.download(ticker, start="2023-01-01", end="2023-12-31") \
    .loc[:, 'Close']
spot_price = data.iloc[-1,0]  # Use the last closing price as the current price

# Estimate volatility (annualized standard deviation of daily returns)
data["Returns"] = data.pct_change()
volatility = data["Returns"].std() * np.sqrt(252)  # Annualized volatility

print(f"Spot Price: {spot_price}")
print(f"Volatility: {volatility}")

[*********************100%***********************]  1 of 1 completed

Spot Price: 139.02532958984375
Volatility: 0.30398176802161114


In [64]:
data

Ticker,GOOGL,Returns
Date,,
2023-01-03,88.695946,NaN
2023-01-04,87.660904,-0.011670
2023-01-05,85.789848,-0.021344
2023-01-06,86.924416,0.013225
2023-01-09,87.601181,0.007786
...,...,...
2023-12-22,140.816757,0.007620
2023-12-26,140.846634,0.000212
2023-12-27,139.702072,-0.008126


In [65]:
# Option parameters
strike_price1 = spot_price * 1.01  # Slightly out-of-the-money
strike_price2 = spot_price * 0.99  # Slightly in-the-money
time_to_expiration = 30 / 365  # 30 days to expiration
risk_free_rate = 0.05 / 100  # 3-month treasury rate (divided by 100)

# Number of contracts
n_contract1 = -100  # Short position in option 1

In [66]:
# Option 1
option1 = EuropeanCall(
    asset_price=spot_price,
    asset_volatility=volatility,
    strike_price=strike_price1,
    time_to_expiration=time_to_expiration,
    risk_free_rate=risk_free_rate
)
print(f"Option 1 - Price: {option1.price}, \nDelta: {option1.delta}, \nGamma: {option1.gamma}")

# Option 2
option2 = EuropeanCall(
    asset_price=spot_price,
    asset_volatility=volatility,
    strike_price=strike_price2,
    time_to_expiration=time_to_expiration,
    risk_free_rate=risk_free_rate
)
print(f"Option 2 - Price: {option2.price}, \nDelta: {option2.delta}, \nGamma: {option2.gamma}")

Option 1 - Price: 4.1951713553974415, 
Delta: 0.4720449901725715, 
Gamma: 0.03284629700099981
Option 2 - Price: 5.537887646031891, 
Delta: 0.5633112207499278, 
Gamma: 0.03251165953556078


In [68]:
# Portfolio Greeks for Option 1
portfolio_greeks = np.array([
    [option1.gamma * abs(n_contract1)],
    [option1.delta * abs(n_contract1)]
])

# Greeks matrix for options 2 and 3
greeks_matrix = np.array([
    [option2.gamma, option1.gamma],
    [option2.delta, option1.delta]
])

# Solve for positions in options 2 and 3
positions = np.linalg.inv(greeks_matrix).dot(portfolio_greeks)

print(f"Positions to neutralize Greeks:\nOption 2: {positions[0][0]}\nOption 3: {positions[1][0]}")

Positions to neutralize Greeks:
Option 2: -1.1368683772161603e-13
Option 3: 100.00000000000006


In [69]:
print(f"Final Portfolio Positions:\n"
      f"Option 1: {n_contract1}\n"
      f"Option 2: {positions[0][0]}\n"
      f"Option 3: {positions[1][0]}")

Final Portfolio Positions:
Option 1: -100
Option 2: -1.1368683772161603e-13
Option 3: 100.00000000000006
